In [8]:
import pandas as pd
import sqlite3
from sqlite3 import Error
pd.options.mode.chained_assignment = None # not to get SettingWithCopyWarning

## Set Variables - Ticker, CIK etc

In [9]:
cik = '0000021344'
ticker = 'KO'

## Function to use

In [10]:
def write_new_data_into_sqltable(db_path=str, df=pd.DataFrame, statement=str, cik=str):
    # create connection
    connection = None
    try:
        connection = sqlite3.connect(db_path)
        print("Connection to SQLite DB successful!")
    except Error as e:
        print(f"The error '{e}' occurred.")
    
    # extend cik to 10 character
    df['cik'] = [str(cik).zfill(10) for cik in df['cik'].values]
    # keep only one CIK in DataFrame
    df = df.loc[df.cik == cik]
    if len(df) == 0:
        print('CIK filter removed every row.')
    # check CIK is in database
    if cik not in pd.read_sql_query("SELECT DISTINCT cik FROM {}".format(statement), connection).values:
        # get newest report date
        new_updated_date = pd.Timestamp(df['end'].values.max()).normalize() + pd.Timedelta(days=1)
        # create direct input tables with strings
        ingest_df = df[pd.read_sql_query("PRAGMA table_info({});".format(statement), connection).name.values].copy()
        for column in ingest_df.columns:
            ingest_df[column].apply(str)
        print('Creation of input database is successful!')
        # write dataframe into table
        ingest_df.to_sql(
            name=statement,
            con=connection,
            if_exists='append',
            index=False)
        print('Data is written into database!')
        # update main table's update column
        cursor = connection.cursor()
        try:
            cursor.execute("UPDATE main SET updated = '{}' WHERE cik = '{}'".format(new_updated_date, cik))
            connection.commit()
            cursor.close()
            print("Main table's updated column freshed successfully! Cursor closed.")
        except Error as e:
            print(f"The error '{e}' occurred")
    else:
        print('Writing into the database is unsuccessful due to existing CIK.')
    # close connection
    connection.close()
    print('Connection closed!')

In [11]:
def add_extra_data_to_sqltable(db_path=str, df=pd.DataFrame, statement=str, cik=str):
    # create connection
    connection = None
    try:
        connection = sqlite3.connect(db_path)
        print("Connection to SQLite DB successful!")
    except Error as e:
        print(f"The error '{e}' occurred.")
    
    # extend cik to 10 characters
    df['cik'] = [str(cik).zfill(10) for cik in df['cik'].values]
    # check CIK is in main datatable
    if cik in pd.read_sql_query("SELECT DISTINCT cik FROM main", connection)['cik'].values:    
        # get last update date relating to company CIK number
        last_update = pd.Timestamp(pd.read_sql_query("SELECT updated FROM main WHERE cik = '{}'".format(cik), connection)['updated'].values[0])
        # filter input df & keep only new data
        filtered_df = df.loc[df.end > last_update]
        filtered_df.reset_index(inplace=True, drop=True)
        if len(filtered_df) != 0 and str(last_update) not in pd.read_sql_query("SELECT end FROM {} WHERE cik = '{}'".format(statement, cik), connection)['end'].values:
            # create direct input tables with strings
            ingest_df = filtered_df[pd.read_sql_query("PRAGMA table_info({});".format(statement), connection).name.values].copy()
            for column in ingest_df.columns:
                ingest_df[column].apply(str)
            print('Creation of input database is successful!')
            # write dataframe into table
            ingest_df.to_sql(
                name=statement,
                con=connection,
                if_exists='append',
                index=False)
            print('Data is written into database!')
            # update main table's update column
            cursor = connection.cursor()
            try:
                new_updated_date = str(pd.to_datetime('today').normalize())
                cursor.execute("UPDATE main SET updated = '{}' WHERE cik = '{}'".format(new_updated_date, cik))
                connection.commit()
                cursor.close()
                print("Main table's updated column freshed successfully! Cursor closed.")
            except Error as e:
                print(f"The error '{e}' occurred")
        else:
            print('There is NO new data!')
    else:
        print('CIK is not in the main data table! Use another function!')
    # close connection
    connection.close()
    print('Connection closed!')

## 0. Create company row in main table & Load X-CHECKED data

In [12]:
data = pd.read_csv('../sec_report_csv/{}_{}_sec_reports.csv'.format(cik, ticker), parse_dates=['end'])
data.head()

,end,form,cik,rev,cost_sale,sale_gen_adm,inpairm,op_income,int_income,int_exp,...,other_ass,tot_ass,short_debt,acc_pay,other_curr_liab,tot_curr_liab,tot_long_debt,other_liab,tot_equity,ticker
0,2015-04-03,10-Q,21344,10711000000,4103000000,4079000000,82000000,2296000000,155000000,447000000,...,4602000000,91016000000,2040000000,8853000000,689000000,26123000000,26087000000,4296000000,28837000000,KO
1,2015-07-03,10-Q,21344,12156000000,4748000000,4204000000,82000000,2535000000,149000000,128000000,...,4517000000,93538000000,2031000000,9997000000,437000000,28852000000,25977000000,4283000000,28418000000,KO
2,2015-10-02,10-Q,21344,11427000000,4577000000,4207000000,82000000,2379000000,155000000,138000000,...,4446000000,93008000000,2692000000,9877000000,383000000,31545000000,25949000000,4194000000,26049000000,KO
3,2015-12-31,10-Q,21344,11427000000,4577000000,4207000000,82000000,2379000000,155000000,138000000,...,4110000000,89996000000,2676000000,9660000000,331000000,26929000000,28311000000,4301000000,25554000000,KO
4,2015-12-31,10-K,21344,44294000000,17482000000,16427000000,4000000,8728000000,613000000,856000000,...,4110000000,89996000000,2676000000,9660000000,331000000,26929000000,28311000000,4301000000,25554000000,KO


## 1. Write new data into SQL table

In [13]:
for statement in ['income_statement', 'balance_sheet']:
    print('Strrting to fill {} table in SQL database!'.format(statement))
    write_new_data_into_sqltable(
        db_path='../database/statements.db',
        df=data,
        statement=statement,
        cik=cik
    )

Strrting to fill income_statement table in SQL database!
Connection to SQLite DB successful!
Creation of input database is successful!
Data is written into database!
Main table's updated column freshed successfully! Cursor closed.
Connection closed!
Strrting to fill balance_sheet table in SQL database!
Connection to SQLite DB successful!
Creation of input database is successful!
Data is written into database!
Main table's updated column freshed successfully! Cursor closed.
Connection closed!


## 2. Update Existing company with newer data

In [14]:
for statement in ['income_statement', 'balance_sheet']:
    print('Strrting to update {} table in SQL database!'.format(statement))
    add_extra_data_to_sqltable(
        db_path='../database/statements.db',
        df=data,
        statement=statement,
        cik=cik
    )

Strrting to update income_statement table in SQL database!
Connection to SQLite DB successful!
There is NO new data!
Connection closed!
Strrting to update balance_sheet table in SQL database!
Connection to SQLite DB successful!
There is NO new data!
Connection closed!
